# National ID Card Recognitionrvices.


The project aims to develop an intelligent system capable of accurately and efficiently recognizing national ID cards using image processing and artificial intelligence technologies. National ID cards are crucial documents used for personal identification and age verification, recognized by both government and private entities.

**What is a CitizenCard?**

A CitizenCard is an official UK ID and proof of age card, recognized by the Home Office and almost all UK retailers and public transport providers, including UK airlines. It facilitates individuals in proving their identity in various everyday situations.

**ID Card Reader Using OpenCV and Tesseract OCR Engine**

In this project, the Open Source Computer Vision Library (OpenCV) is used for image processing, offering a wide range of tools and techniques for effective image and video handling. Additionally, the Tesseract OCR engine, a powerful and free open-source tool for text recognition, is employed to convert text from images into digital format.

By integrating these technologies, the project aims to build a system capable of accurately reading and analyzing national ID cards, enhancing the efficiency of identity verification processes in various practical applications, such as security checks and access to services.

## Import libraries


In [ ]:
import cv2
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Image Preprocessing

In [ ]:
# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to preprocess the image
def preprocess_image(image_path):
    # Load image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    
    # Apply GaussianBlur to reduce noise and improve edge detection
    gray = cv2.medianBlur(gray, 5)

     # Apply thresholding to get a binary image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    
    # Further noise reduction using median blur
    median = cv2.medianBlur(thresh, 3)
    
    # Morphological transformations to improve the text area
    kernel = np.ones((2, 2), np.uint8)
    dilation = cv2.dilate(median, kernel, iterations=1)
    erosion = cv2.erode(dilation, kernel, iterations=1)

    # Sharpen the image to enhance text clarity
    sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened = cv2.filter2D(erosion, -1, sharpen_kernel)
    
    # Edge detection
    edges = cv2.Canny(erosion, 30, 150)

    return image, gray, sharpened, edges

#### Insight:

- **Setting Up Tesseract Path:** The path to the Tesseract OCR executable is specified to allow Tesseract to be used for text recognition.

- **Image Preprocessing Function:** The preprocess_image function takes an image path as input and processes the image to enhance text recognition.

- **Loading and Grayscale Conversion:** The image is loaded using cv2.imread and then converted to a grayscale image to simplify further processing.

- **Contrast Enhancement:** CLAHE is applied to improve the contrast of the grayscale image, making text regions more distinct.

- **Noise Reduction:** Median blur is applied twice (before and after thresholding) to reduce noise and improve the clarity of edges and text.

- **Thresholding:** Converts the image to a binary image, where text areas become white on a black background, enhancing text detection.

- **Morphological Transformations:** Dilation and erosion are used to refine the text areas, making them more distinguishable.

- **Sharpening:** The image is sharpened using a filter to enhance text clarity further.

- **Edge Detection:** Canny edge detection is applied to identify the edges in the image, which can help in further text extraction processes.

The function returns the original image and several processed versions, each highlighting different aspects crucial for effective OCR.

## Find contours and extract the ID card

In [ ]:
# Function to find contours and extract the ID card
def extract_id_card(edged, image):
    # Find contours
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    screenCnt = None
    
    # Loop over contours to find the ID card
    for c in contours:
        # Approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # If the approximated contour has four points, assume it is the ID card
        if len(approx) == 4:
            screenCnt = approx
            break
   
    # Ensure that we have found a contour with four points
    if screenCnt is None:
        raise ValueError("Could not find an ID card contour")
 
    # Transform the perspective to get a top-down view of the ID card
    pts = screenCnt.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    
    return warp

#### Insight:

1- **Finding Contours:** The function starts by finding contours in the provided edge-detected image using cv2.findContours. Contours are then sorted by area in descending order to prioritize larger contours.

2- **Contour Approximation:** The function loops through the sorted contours to approximate their shapes. If a contour with four points is found, it is assumed to be the ID card.

3- **Contour Validation:** If no contour with four points is found, the function raises an error indicating the ID card could not be detected.

4- **Perspective Transformation:** Once the ID card contour is identified, the function transforms the perspective to get a top-down view of the ID card. This involves:
- Reshaping and reordering the contour points.
- Calculating the maximum width and height of the transformed card.
- Creating a destination array for the transformed image.
- Computing the perspective transformation matrix.
- Applying the perspective transformation to obtain a corrected view of the ID card.

5- **Output:** The function returns the warped (corrected perspective) image of the ID card, which can then be used for further processing, such as OCR.

This process ensures that the ID card is accurately extracted from the image, regardless of its initial orientation or perspective.

# Text Recognition with Tesseract OCR

In [ ]:
# Function to extract text using Tesseract OCR
def extract_text(image):
    # Convert the image to RGB (for Tesseract)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Use Tesseract to do OCR on the  image
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(rgb_image, lang='eng', config=custom_config)

    return text

#### Insight:

1- **Image Conversion:** The function first converts the input image from BGR to RGB color space using cv2.cvtColor. This step is necessary because Tesseract OCR expects images in RGB format.

2- **Tesseract OCR Configuration:** The function sets custom configuration options for Tesseract OCR:
- --oem 3: Specifies the OCR Engine Mode (OEM), where mode 3 means using both the legacy engine and the LSTM (Long Short-Term Memory) neural network engine.
- --psm 6: Specifies the Page Segmentation Mode (PSM), where mode 6 assumes a single uniform block of text.

3- **Text Extraction:** The function uses pytesseract.image_to_string to perform OCR on the RGB image with the specified configuration and language set to English ('eng').

4- **Return Value:** The function returns the extracted text as a string.

This function effectively converts an image to a format suitable for OCR and extracts text from it using Tesseract, allowing for automated text recognition from images.

# Data Structuring

In [ ]:
# Function to structure the extracted text into a pandas DataFrame
def structure_text(text):
    # Split the extracted text into lines and clean up the lines
    lines = text.split('\n')
    lines = [line.strip() for line in lines if line.strip()]
    
    # Extract the relevant information
    info = {
        "card_type": None,
        "Name": None,
        "DoB": None,
        "Expires": None,
        "Number of ID": None,
        "18 on": None,
        "Age range": None
    }

    # Improved parsing logic
    try:
        info["card_type"] = lines[0]  # The first line is the card type
        
        for i, line in enumerate(lines):
            if line.lower().startswith("name"):
                info["Name"] = lines[i + 1] + " " + lines[i + 2]
            elif 'Dob' in line or 'DoB' in line:
                info["DoB"] = lines[i + 1].strip('is PA : . PAss’ Bee re')
            elif line.lower().startswith("expires on"):
                info["Expires"] = lines[i + 1]
            elif '5843' in line:
                info["Number of ID"] = line.strip('Ny,. ) of ')
            else:
                info["18 on"] = ' '.join(line.split()[:3])
                info["Age range"] = ' '.join(line.split()[3:6])
    except IndexError as e:
        print(f"Error parsing lines: {e}")
        print(f"Lines extracted: {lines}")

    
    # Create a DataFrame
    df = pd.DataFrame([info])
    return df

#### Insight:

The structure_text function takes text extracted by OCR and structures it into a Pandas DataFrame. It first cleans and splits the text into lines. Then, it initializes a dictionary to hold specific ID card information such as the card type, name, date of birth, expiration date, ID number, and age-related details. The function attempts to parse these details from the cleaned text lines using conditional checks and assigns them to the dictionary. If an error occurs during parsing, it prints an error message. Finally, it converts the dictionary into a Pandas DataFrame for structured representation of the extracted information. This process ensures organized and easily accessible data from the OCR output.







## Main function

In [ ]:
# Main function to run the entire pipeline
def process_id_card(image_path):
    # Preprocess the image
    image, gray, sharpened, edged = preprocess_image(image_path)
    # Extract the ID card from the image
    id_card = extract_id_card(edged, image)
    #Extract text from the ID card using Tesseract OCR
    text = extract_text(id_card)
    #Structure the text into a pandas DataFrame
    df = structure_text(text)
    return df

# Testing and Validation

### Test using ID cards for '16-17' for young people over 16


In [ ]:
# Test the pipeline with a sample ID card image
image_path = 'uk-id-card-for-16-17s.png'
df = process_id_card(image_path)
df

In [ ]:
# Display the original image and the processed ID card
image, gray, sharpened, edges = preprocess_image(image_path)
id_card = extract_id_card(edges, sharpened)

plt.figure(figsize=(15, 5))
plt.subplot(1, 4, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.subplot(1, 4, 2)
plt.title('Gray Image')
plt.imshow(gray, cmap='gray')
plt.subplot(1, 4, 3)
plt.title('Binary Image')
plt.imshow(sharpened, cmap='gray')
plt.subplot(1, 4, 4)
plt.title('Edges')
plt.imshow(edges, cmap='gray')
plt.show()

### Test using ID cards for 'Under 16' for children


In [ ]:
# Test the pipeline with a sample ID card image
image_path = 'uk-id-card-for-under16s.jpg'
df = process_id_card(image_path)
df

In [ ]:
# Display the original image and the processed ID card
image, gray, sharpened, edges = preprocess_image(image_path)
id_card = extract_id_card(edges, sharpened)

plt.figure(figsize=(15, 5))
plt.subplot(1, 4, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.subplot(1, 4, 2)
plt.title('Gray Image')
plt.imshow(gray, cmap='gray')
plt.subplot(1, 4, 3)
plt.title('Binary Image')
plt.imshow(sharpened, cmap='gray')
plt.subplot(1, 4, 4)
plt.title('Edges')
plt.imshow(edges, cmap='gray')
plt.show()

--------------------------------------------------------